### **Repo Selection**

In [128]:
# TODO: Discuss idea with Professor
# Q. What high prompt-density repositories should we be looking at? And, for which parsers?

# USING DEFAULT PARSER for now
import json
with open('../parse_data/repo_to_prompts.json', 'r') as f:
    data = json.load(f)

all_prompts = []
high_density_repo_prompts = []
for repo in data:
    all_prompts.extend(data[repo])
    if len(data[repo]) > 10:
        high_density_repo_prompts.extend(data[repo])

print(len(all_prompts))
print(len(high_density_repo_prompts))

1127
577


Saving prompts to files so that cspell can check them.

In [130]:
import os
import subprocess

root_dir = 'repos_temp'
# Make repo_temp
if not os.path.exists(root_dir):
    os.makedirs(root_dir)
    for repo in data:
        filename = f"{repo}.py"
        path = os.path.join(root_dir, filename)
        with open(path, 'w') as f:
            f.writelines(data[repo])

In [153]:
import subprocess

# file_to_check = "repos_temp/836304831~langchain-anal.py"
subprocess.run(["cspell", "repos_temp/836304831~langchain-anal.py"])

CompletedProcess(args=['cspell', 'repos_temp/836304831~langchain-anal.py'], returncode=0)

### **Text Analysis**

In [88]:
import pandas as pd
df = pd.DataFrame(high_density_repo_prompts)
df.columns = ['prompts']

df.to_csv('high_density_repo_prompts.csv', index=False)

count = 0
for prompt in high_density_repo_prompts:
    if "assistant" in prompt.lower() or "assistent" in prompt.lower():
        count += 1
        # print(prompt)
count

51

### ❌ **Spell Check** (FAILED) (WIP)

In [110]:
import spacy
from textblob import TextBlob, Word
from spacy.language import Language
import re

misspelled_freq = {}
@Language.component("spellcheck_component")
def spellcheck_component(doc):
    # Use TextBlob to find misspelled words
    # Initialize TextBlob with the text of the entire spaCy document
    blob = TextBlob(doc.text)

    for word in blob.words:
        # Use TextBlob's Word object for spellchecking
        w = Word(word)
        # If the word is misspelled, TextBlob will have a suggestion with higher confidence
        if w.correct() == word:
            continue
        # Add the misspelled word to our dictionary with its frequency
        misspelled_freq[word] = misspelled_freq.get(word, 0) + 1
            
    # for word in misspelled:
    #     misspelled_freq[word] = misspelled_freq.get(word, 0) + 1

    # Return the doc following spaCy's pipeline component requirements
    return doc

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Add spell-checking component to the pipeline
nlp.add_pipe("spellcheck_component")

text = "The quick brwn fox jmps over the lze dog."

# Process the text with spaCy and check spelling
nlp(text)

count = 0
for prompt in high_density_repo_prompts:
    modified_string = re.sub(r'((\{+)(\s*)(.*?)(\s*)(\}+))', ' ', prompt)  # remove {variable}
    modified_string = re.sub(r'\\n', ' ', modified_string)  # to avoid "\nTonight"
    modified_string = re.sub(r"[^\w']+", " ", modified_string)  # to avoid "good-morning" but words like keep "don't"
    modified_string = re.sub(r"_+", " ", modified_string)  # to avoid "______" or "good__morning"
    doc = nlp(modified_string)
    count += 1
    if count % 10 == 0:
        print(count)


print(sorted(misspelled_freq.items(), key=lambda x: x[1], reverse=True))

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
[('AI', 289), ("'s", 274), ("n't", 172), ('If', 172), ('This', 91), ('Let', 84), ('Do', 77), ('We', 72), ('pl', 72), ('json', 68), ('template', 64), ('Make', 63), ('No', 60), ('url', 53), ('llm', 48), ('api', 48), ('What', 44), ('openai', 43), ('ui', 42), ('Here', 41), ("'m", 39), ('inputs', 38), ('Tool', 37), ('Inputs', 36), ('OF', 34), ('Get', 33), ('blog', 33), ('How', 32), ('Modal', 32), ('python', 30), ('My', 30), ('Mora', 30), ('Rivera', 30), ("'ui", 28), ('Pay', 27), ('doc', 27), ('website', 27), ('func', 22), ('tweet', 22), ('str', 22), ('implemented', 22), ("'S", 22), ('Task', 21), ('Be', 21), ('qubits', 21), ('formatting', 20), ('annotation', 20), ('interfaces', 20), ('UX', 20), ('integrations', 20), ('loader', 19), ('Load', 19), ('schema', 19), ('单位', 19), ('g

### 🧩 **Searching Patterns** (WIP)

In [91]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)
# Add match ID "HelloWorld" with no callback and one pattern
patterns = [
    [{"LOWER": "assistant"}],
]

matcher.add("GPPT-3", patterns)

def find_pattern(text, patterns):
    doc = nlp(text)
    matches = matcher(doc)
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]  # Get string representation
        span = doc[start:end]  # The matched span
        print(match_id, string_id, start, end, span.text)

    return matches

count = 0
for p in high_density_repo_prompts:
    if find_pattern(p, patterns):
        count += 1
print(count)

14194311245358984270 GPPT-3 70 71 assistant
14194311245358984270 GPPT-3 84 85 assistant
14194311245358984270 GPPT-3 73 74 assistant
14194311245358984270 GPPT-3 87 88 assistant
14194311245358984270 GPPT-3 8 9 assistant
14194311245358984270 GPPT-3 17 18 assistant
14194311245358984270 GPPT-3 30 31 assistant
14194311245358984270 GPPT-3 113 114 assistant
14194311245358984270 GPPT-3 264 265 assistant
14194311245358984270 GPPT-3 13 14 assistant
14194311245358984270 GPPT-3 25 26 assistant
14194311245358984270 GPPT-3 359 360 assistant
14194311245358984270 GPPT-3 7 8 assistant
14194311245358984270 GPPT-3 27 28 Assistant
14194311245358984270 GPPT-3 5 6 assistant
14194311245358984270 GPPT-3 7 8 assistant
14194311245358984270 GPPT-3 6 7 assistant
14194311245358984270 GPPT-3 7 8 assistant
14194311245358984270 GPPT-3 7 8 assistant
14194311245358984270 GPPT-3 7 8 assistant
14194311245358984270 GPPT-3 7 8 assistant
14194311245358984270 GPPT-3 7 8 assistant
14194311245358984270 GPPT-3 60 61 Assistant
14